In [7]:
import config
import numpy as np
import Test_without_Training

def get_data(K1, K2, subj1, subj2):
    trainer1 = Test_without_Training.TremorModelTrainer(config, subject=subj1)
    X1_train, y1_train, X1_test, y1_test = trainer1.return_K_data(K=K1)

    trainer2 = Test_without_Training.TremorModelTrainer(config, subject=subj2)
    X2_train, y2_train, X2_test, y2_test = trainer2.return_K_data(K=K2)

    # Combine and shuffle training data from both subjects
    X_train = np.concatenate([X1_train, X2_train], axis=0)
    y_train = np.concatenate([y1_train, y2_train], axis=0)
    idx = np.random.permutation(len(X_train))
    X_train, y_train = X_train[idx], y_train[idx]

    return {
        "trainer": trainer2,  # just pick one to call train_multiple_dataset
        "X1_train": X1_train, "y1_train": y1_train,
        "X2_train": X2_train, "y2_train": y2_train,
        "X_train": X_train,   "y_train": y_train,
        "X1_test": X1_test,   "y1_test": y1_test,
        "X2_test": X2_test,   "y2_test": y2_test
    }

def get_results(K1, K2, subj1, subj2):
    data = get_data(K1, K2, subj1, subj2)
    trainer = data["trainer"]

    # Train with mixed, only subj2, and only subj1
    acc1, model_mix = trainer.train_multiple_dataset(data["X_train"], data["y_train"], data["X1_test"], data["y1_test"])
    acc2, model_s2  = trainer.train_multiple_dataset(data["X2_train"], data["y2_train"], data["X1_test"], data["y1_test"])
    acc3, model_s1  = trainer.train_multiple_dataset(data["X1_train"], data["y1_train"], data["X1_test"], data["y1_test"])

    # Evaluate all models on subj2
    _, acc_mix_s2 = model_mix.evaluate(data["X2_test"], data["y2_test"], verbose=0)
    _, acc_s2_s2  = model_s2.evaluate(data["X2_test"], data["y2_test"], verbose=0)
    _, acc_s1_s2  = model_s1.evaluate(data["X2_test"], data["y2_test"], verbose=0)

    return [acc1, acc2, acc3], [acc_mix_s2, acc_s2_s2, acc_s1_s2]

def print_results(acc_H, acc_X):
    print(f"Trained with Subjects H and X / Tested on H (Inter-session):     {acc_H[0]:.2f}%")
    print(f"Trained with Subject H        / Tested on H (Inter-session):     {acc_H[2]:.2f}%")
    print(f"Trained with Subject X        / Tested on H (Inter-subject):     {acc_H[1]:.2f}%\n")

    print(f"Trained with Subjects H and X / Tested on X (Inter-session):     {acc_X[0]*100:.2f}%")
    print(f"Trained with Subject X        / Tested on X (Inter-session):     {acc_X[1]*100:.2f}%")
    print(f"Trained with Subject H        / Tested on X (Inter-subject):     {acc_X[2]*100:.2f}%")


In [2]:
acc_H_base, acc_X_base = get_results(K1=1, K2=1, subj1="Hunmin", subj2="Xianyu")
acc_H, acc_X = get_results(K1=28, K2=6, subj1="Hunmin", subj2="Xianyu")

Dataset 1/34 - Session Exp_2025-05-27/E8331D05289A/
Dataset 2/34 - Session Exp_2025-06-18/E9AD0E7DCC2B/
Dataset 3/34 - Session Exp_2025-06-20-v1/E9AD0E7DCC2B/
Dataset 4/34 - Session Exp_2025-06-20-v2/E9AD0E7DCC2B/
Dataset 5/34 - Session Exp_2025-06-20-v3/E9AD0E7DCC2B/
Dataset 6/34 - Session Exp_2025-06-20-v4/E9AD0E7DCC2B/
Dataset 7/34 - Session Exp_2025-06-20-v5/E9AD0E7DCC2B/
Dataset 8/34 - Session Exp_2025-06-20-v6/E9AD0E7DCC2B/
Dataset 9/34 - Session Exp_2025-06-20-v7/E9AD0E7DCC2B/
Dataset 10/34 - Session Exp_2025-06-20-v8/E9AD0E7DCC2B/
Dataset 11/34 - Session Exp_2025-06-23-v1/E9AD0E7DCC2B/
Dataset 12/34 - Session Exp_2025-06-23-v2/E9AD0E7DCC2B/
Dataset 13/34 - Session Exp_2025-06-23-v3/E9AD0E7DCC2B/
Dataset 14/34 - Session Exp_2025-06-23-v4/E9AD0E7DCC2B/
Dataset 15/34 - Session Exp_2025-06-24-v1/E9AD0E7DCC2B/
Dataset 16/34 - Session Exp_2025-06-24-v2/E9AD0E7DCC2B/
Dataset 17/34 - Session Exp_2025-06-24-v3/E9AD0E7DCC2B/
Dataset 18/34 - Session Exp_2025-06-24-v4/E9AD0E7DCC2B/
Dataset

In [8]:
print_results(acc_H_base, acc_X_base)
print("\n")
print_results(acc_H, acc_X)

Trained with Subjects H and X / Tested on H (Inter-session):     76.00%
Trained with Subject H        / Tested on H (Inter-session):     78.24%
Trained with Subject X        / Tested on H (Inter-subject):     68.12%

Trained with Subjects H and X / Tested on X (Inter-session):     71.18%
Trained with Subject X        / Tested on X (Inter-session):     69.94%
Trained with Subject H        / Tested on X (Inter-subject):     58.46%


Trained with Subjects H and X / Tested on H (Inter-session):     90.20%
Trained with Subject H        / Tested on H (Inter-session):     91.02%
Trained with Subject X        / Tested on H (Inter-subject):     77.17%

Trained with Subjects H and X / Tested on X (Inter-session):     67.11%
Trained with Subject X        / Tested on X (Inter-session):     70.16%
Trained with Subject H        / Tested on X (Inter-subject):     51.17%
